In [27]:
import numpy as np

def mdpgen(S, A):
    """
    Generate a random MDP with S states and A actions.

    Returns:
        P    (S, S, A) array  : transition kernels P(s' | s, a)
        R    (S, A)     array : rewards R(s, a) in [0, 10)
        Ppi  (S, S)     array : transition matrix under uniform policy
        p    (S,)       array : stationary distribution of Ppi
    """
    # 1) Random transition kernels, then normalize rows
    P = np.random.rand(S, S, A)
    for j in range(A):
        for i in range(S):
            P[i, :, j] /= P[i, :, j].sum()

    # 2) Uniform behavior policy transition matrix
    Ppi = P.sum(axis=2) / A

    # 3) Stationary distribution: left eigenvector of Ppi for eigenvalue 1
    #    which is the right eigenvector of Ppi.T
    eigvals, eigvecs = np.linalg.eig(Ppi.T)
    idx = np.argmin(np.abs(eigvals - 1.0))
    p = np.real(eigvecs[:, idx])
    # enforce non-negativity up to a sign
    p *= np.sign(p)
    p /= p.sum()

    # 4) Random reward function
    R = np.random.rand(S, A)

    return P, R, Ppi, p


In [28]:
S, A, gamma, T = 10, 5, 0.5, 250000
P, R, Ppi, p = mdpgen(S, A)
def value_iteration(P, R, gamma, T):
    """
    Run value iteration for T steps to compute optimal Q-function.
    Returns:
        Q     (S, A)   array : estimated state-action values
        Err   (T,)     array : infinity-norm Bellman residual at each iteration
    """
    S, _, A = P.shape
    Q = np.zeros((S, A))
    Err = np.zeros(T)
    for t in range(T):
        Q_old = Q.copy()
        V = Q_old.max(axis=1)  # state-value function
        # Bellman optimality update vectorized
        Q = R + gamma * np.tensordot(P, V, axes=([1], [0]))
        Err[t] = np.max(np.abs(Q - Q_old))
    return Q, Err



In [29]:
# Generate MDP and optimal Q-values
#P, R = mdpgen(S, A)
Qopt,_ = value_iteration(P, R, gamma, T=250000)
#Qopt = np.array(Qopt)

In [30]:
for name, val in [('P', P), ('R', R), ('p', p), ('Qopt', Qopt), ('gamma', gamma), ('S', S), ('A', A)]:
    print(f"{name}: type={type(val)}, shape={getattr(val, 'shape', 'N/A')}")


P: type=<class 'numpy.ndarray'>, shape=(10, 10, 5)
R: type=<class 'numpy.ndarray'>, shape=(10, 5)
p: type=<class 'numpy.ndarray'>, shape=(10,)
Qopt: type=<class 'numpy.ndarray'>, shape=(10, 5)
gamma: type=<class 'float'>, shape=N/A
S: type=<class 'int'>, shape=N/A
A: type=<class 'int'>, shape=N/A


In [31]:
np.savez('Qlearn_robust_fed_q_data_new.npz',
         P=P,
         R=R,
         p=p,
         Qopt=Qopt,
         gamma=gamma,
         S=np.int32(S),
         A=np.int32(A))

In [32]:
import numpy as np

def median_of_means(X, P_groups):
    X = np.asarray(X, dtype=float)
    n = len(X)
    P = max(1, min(int(P_groups), n))
    if P == 1:
        return float(X.mean())
    idx = np.random.permutation(n)
    blocks = np.array_split(X[idx], P)
    means = np.array([b.mean() for b in blocks if b.size > 0], dtype=float)
    return float(np.median(means))


In [33]:
def choose_mom_params(S, A, T, M, epsilon, delta):
    bar_delta = max(delta / (S * A * T), 1e-12)
    raw_P = int(np.ceil(8 * epsilon * M + (256.0 / 7.0) * np.log(2.0 / bar_delta)))
    return max(1, min(raw_P, M))  # 1 ≤ P ≤ M

def empirical_transition_estimates(samples, S, A, H):
    counts = np.zeros((S, S, A))
    for s, a, s_next in samples:
        counts[s_next, s, a] += 1
    # Add a small value to prevent division by zero if a state-action pair was not visited
    P_hat = counts / (np.sum(counts, axis=0) + 1e-9)
    return P_hat

def agent_epoch_update(Q, R, P_hat, gamma):
    S, _, A = P_hat.shape
    d = np.zeros((S, A))
    V = Q.max(axis=1)
    for s in range(S):
        for a in range(A):
            # P_hat is (S, S, A), we need P_hat[:, s, a] which is (S,)
            d[s, a] = R[s, a] + gamma * np.dot(P_hat[:, s, a], V)
    return d

def choose_K(M, T, gamma, c1=10):
    return int(np.ceil(c1 * np.log(M * T) / (1 - gamma)))

def compute_alpha(M, T, gamma, K):
    return np.log(M * T) / ((1 - gamma) * K)

In [34]:
# --- Return (Q, residual_trace) and sample with correct indexing P_env[s, :, a] ---
def robust_fed_q(S, A, M, T, gamma, epsilon, delta, P_env, R_env, alpha, K):
    H = max(1, T // K)
    Q = np.zeros((S, A))
    residual_trace = []

    for k in range(K):
        directions = np.zeros((M, S, A))
        for i in range(M):
            samples = []
            # Collect H samples for each (s,a)
            for s in range(S):
                for a in range(A):
                    for _ in range(H):
                        p_dist = P_env[s, :, a]              # P(s' | s,a)  <-- FIXED
                        s_next = np.random.choice(S, p=p_dist)
                        samples.append((s, a, s_next))

            P_hat = empirical_transition_estimates(samples, S, A, H)
            d = agent_epoch_update(Q, R_env, P_hat, gamma)

            if np.random.rand() < epsilon:
                d = d + np.random.laplace(scale=10.0, size=d.shape)

            directions[i] = d

        P_mom = choose_mom_params(S, A, T, M, epsilon, delta)
        for s in range(S):
            for a in range(A):
                Q[s, a] = (1 - alpha) * Q[s, a] + alpha * median_of_means(directions[:, s, a], P_mom)

        # track ∞-norm Bellman residual under TRUE env
        V = Q.max(axis=1)
        TQ_true = np.zeros_like(Q)
        for s in range(S):
            for a in range(A):
                TQ_true[s, a] = R_env[s, a] + gamma * np.dot(P_env[s, :, a], V)  # <-- consistent with mdpgen
        residual_trace.append(np.max(np.abs(TQ_true - Q)))

    return Q, np.array(residual_trace)


In [ ]:
import matplotlib.pyplot as plt
epsilon = 0.001
delta = 0.05
agent_list = [3, 10, 50, 500]

traces = {}
for M in agent_list:
    K = choose_K(M, T, gamma)
    alpha = compute_alpha(M, T, gamma, K)
    Q_final, res_trace = robust_fed_q(S, A, M, T, gamma, epsilon, delta, P, R, alpha, K)
    traces[M] = res_trace

plt.figure(figsize=(12, 8))
for M in agent_list:
    plt.semilogy(traces[M], label=fr'$M={M}$', linewidth=4)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel(r'$\mathbf{E_t}$', fontsize=14)
plt.grid(True, linewidth=3, which="both")
plt.legend(title='Agents', fontsize=12)
plt.tight_layout()
plt.show()